In [1]:
import cv2
import numpy as np
from ultralytics import YOLO
from detection import *

In [2]:
MODEL_PATH = r"..\runs\detect\train3\weights\best.pt"

### Get BBOX from YOLO

In [3]:
def detect_xo(frame, model):
    """
    детекция крестиков и ноликов для картинки/фрейма 
    """
    results = model.predict(source=frame, conf=0.5, show=False, verbose=False)  # Adjust `conf` if necessary
    detected_objects = []

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
            label = result.names[int(box.cls)]
            confidence = float(box.conf)

            # Determine the cell that contains this object
            detected_objects.append({
                'label': label,
                'confidence': confidence,
                'bbox': (x1, y1, x2, y2)
            })
    return detected_objects

def load_yolo_model(model_path):
    return YOLO(model_path)

def process_video(model_path, show_image=False):
    model = load_yolo_model(model_path)

    # 0 если лайв вебкамера 
    video = cv2.VideoCapture("../videos/test_video.mp4")
    
    # Get grid cells from the first frame for reference
    ret, first_frame = video.read()
    if not ret:
        print("Error reading video.")
        return

    # Loop over frames
    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            break
        
        detected_objects = detect_xo(frame, model)

        if show_image:
            cv2.imshow('Real-Time Tic Tac Toe', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break  # Press 'q' to exit the loop

    video.release()
    cv2.destroyAllWindows()

### Tic Tac Toe Video Algorithm

In [4]:
# Helper function to find cell containing the center of the bounding box
def find_cell_for_object(center, cells):
    for index, cell in enumerate(cells):
        if cell['x0'] <= center[0] <= cell['x1'] and cell['y0'] <= center[1] <= cell['y1']:
            return index  # Return the cell index if the center is within cell bounds
    return None

# Function to detect X/O and identify corresponding cell
def detect_xo_and_identify_cells(frame, model, cells):
    results = model.predict(source=frame, conf=0.5, show=False, verbose=False)  # Adjust `conf` if necessary
    detected_objects = []

    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
            label = result.names[int(box.cls)]
            confidence = float(box.conf)

            # Determine the cell that contains this object
            cell_index = find_cell_for_object((center_x, center_y), cells)
            
            if cell_index is not None:
                detected_objects.append({
                    'label': label,
                    'confidence': confidence,
                    'cell_index': cell_index,
                    'bbox': (x1, y1, x2, y2)
                })
    return detected_objects


In [5]:
# Modify get_original_image_cells to accept an image array instead of path
def get_original_image_cells(image, resize_dim=(600, 600), show_image=False):
    """
    Detect grid and cells in an image (either as a path or image array) and return cell coordinates.
    """
    # Detect the grid and get the perspective transformation matrix
    cells = detect_grid_and_cells(image_path=None, image=image, resize_dim=resize_dim, show_image=show_image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    contours, _ = cv2.findContours(cv2.adaptiveThreshold(gray, 255, 1, 1, 11, 2), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    max_area, best_cnt = 0, None
    for c in contours:
        area = cv2.contourArea(c)
        if area > 10000 and area > max_area:
            max_area = area
            best_cnt = c

    # Find four corners of the grid for transformation
    try: 
        epsilon = 0.02 * cv2.arcLength(best_cnt, True)
        approx = cv2.approxPolyDP(best_cnt, epsilon, True)
        pts = np.array([point[0] for point in approx], dtype="float32")
        rect = np.zeros((4, 2), dtype="float32")
        s, diff = pts.sum(axis=1), np.diff(pts, axis=1)
        rect[0], rect[2] = pts[np.argmin(s)], pts[np.argmax(s)]
        rect[1], rect[3] = pts[np.argmin(diff)], pts[np.argmax(diff)]
        
        # Compute perspective transform and inverse transform matrices
        dst_pts = np.array([[0, 0], [resize_dim[0], 0], [resize_dim[0], resize_dim[1]], [0, resize_dim[1]]], dtype="float32")
        M = cv2.getPerspectiveTransform(rect, dst_pts)
        M_inv = cv2.getPerspectiveTransform(dst_pts, rect)

        # Apply perspective transform to get cells in transformed space
        warped_image = apply_perspective_transform(image, rect, resize_dim[0], resize_dim[1])
        transformed_cells = divide_into_cells((0, 0, resize_dim[0], resize_dim[1]), warped_image)

        # Transform cells back to original coordinates using M_inv
        original_cells = []
        for cell in transformed_cells:
            top_left = np.dot(M_inv, np.array([cell['x0'], cell['y0'], 1]))
            bottom_right = np.dot(M_inv, np.array([cell['x1'], cell['y1'], 1]))
            original_cells.append({
                'x0': int(top_left[0] / top_left[2]),
                'y0': int(top_left[1] / top_left[2]),
                'x1': int(bottom_right[0] / bottom_right[2]),
                'y1': int(bottom_right[1] / bottom_right[2])
            })

        return original_cells
    
    except:
        raise ValueError("grid not found")

In [36]:
def send_move_to_robot(robot_move):
    pass

from typing import Dict, Tuple
def convert_ai_move_for_robot(move: Tuple[int, int], cells: Dict): 
    cell_index = move[0] * 3 + move[1]
    x_center = (cells[cell_index]['x0'] + cells[cell_index]['x1']) / 2
    y_center = (cells[cell_index]['y0'] + cells[cell_index]['y1']) / 2
    return x_center, y_center

In [16]:
move = (1, 0)
move[0] * 3 + move[1]

3

In [32]:
import cv2

def draw_cells(frame, cells, color=(0, 255, 0), thickness=2, show_index=True):
    """
    Draws rectangles on the frame for each cell in cells and optionally labels them with their index.

    :param frame: The image/frame on which to draw.
    :param cells: A list of dictionaries with keys 'x0', 'y0', 'x1', 'y1'.
    :param color: Tuple representing the BGR color of the rectangle.
    :param thickness: Thickness of the rectangle border.
    :param show_index: Boolean indicating whether to draw the index label.
    :return: The frame with drawn cells and indices.
    """
    for idx, cell in enumerate(cells):
        top_left = (cell['x0'], cell['y0'])
        bottom_right = (cell['x1'], cell['y1'])
        cv2.rectangle(frame, top_left, bottom_right, color, thickness)
        
        if show_index:
            # Calculate the position for the index label (top-left corner with some padding)
            text_position = (cell['x0'] + 5, cell['y0'] + 25)
            # Define the font, scale, color, and thickness
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.7
            text_color = (0, 0, 255)  # Red color for visibility
            text_thickness = 2
            # Put the index text on the frame
            cv2.putText(frame, str(idx), text_position, font, font_scale, text_color, text_thickness, cv2.LINE_AA)
    
    return frame


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from detection import *  # Import helper functions from your file
from collections import defaultdict
import time
from gholb import tictactoe as ttt 

# Load YOLO model
def load_yolo_model(model_path):
    return YOLO(model_path)

# Track the position stability for detected objects
def track_positions(detected_objects, frame_time, stable_positions, duration=2):
    confirmed_moves = []
    for obj in detected_objects:
        cell_index = obj['cell_index']
        label = obj['label']
        
        if (label, cell_index) in stable_positions:
            last_seen_time = stable_positions[(label, cell_index)]
            if frame_time - last_seen_time >= duration:
                confirmed_moves.append(obj)
        else:
            stable_positions[(label, cell_index)] = frame_time
    
    # Remove entries not seen in this frame
    stable_positions = {key: stable_positions[key] for key in stable_positions if key in {(obj['label'], obj['cell_index']) for obj in detected_objects}}
    return confirmed_moves, stable_positions

# Process live video feed and detect objects
def process_live_video(video_path, model_path, show_image=False, user='X'):

    ai_turn = True
    if user == 'X':
        ai_turn = False

    model = load_yolo_model(model_path)

    # Open live video feed (0 is usually the default webcam)
    video = cv2.VideoCapture(video_path)
    
    # Get grid cells from the first frame for reference
    ret, first_frame = video.read()
    if not ret:
        print("Error reading video.")
        return
    
    # NOTE those are bboxes for now
    original_cells = get_original_image_cells(first_frame, show_image=show_image) # order is correct

    if show_image:
        first_frame_with_cells = draw_cells(first_frame.copy(), original_cells, color=(0, 255, 0), thickness=2)
        cv2.imshow('Original Cells Verification', first_frame_with_cells)
        print("Press any key to continue...")
        cv2.waitKey(0)

    # Initialize stable positions with the first frame detections
    initial_detections = detect_xo_and_identify_cells(first_frame, model, original_cells)
    frame_time = time.time()
    current_positions = {(obj['label'], obj['cell_index']): frame_time for obj in initial_detections}
    board = initial_state()

    # Loop over frames
    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            break
        frame_time = time.time()  # Current frame time
        
        detected_objects = detect_xo_and_identify_cells(frame, model, original_cells)
        confirmed_moves, current_positions = track_positions(
            detected_objects, 
            frame_time, 
            stable_positions=current_positions, 
            duration=2
        )

        print(confirmed_moves, current_positions)
        
        if confirmed_moves:
            board = update_board(board, confirmed_moves)
            # Display the updated board and moves if needed
            print("Board updated:", board)
            
            # Here we call Gholibs function to get next move. This will take some time. 
            game_over = ttt.terminal(board)
            player = ttt.player(board)

            print(f'next move {player}')

            if game_over:
                winner = ttt.winner(board)
                if winner is None:
                    title = f"Game Over: Tie."
                else:
                    title = f"Game Over: {winner} wins."
                print(title)
                break
            elif user == player:
                title = f"Play as {user}"
            else:
                title = f"Computer thinking..."
            print(title)

            if user != player and not game_over:
                print('robot enter')
                if ai_turn:
                    print("robot maces moves")
                    time.sleep(0.5)
                    move = ttt.minimax(board) # это (i, j) - индексы ячейки
                    board = ttt.result(board, move)

                    # Send cell center to robot
                    robot_move = convert_ai_move_for_robot(move)
                    send_move_to_robot(robot_move)

                    ai_turn = False
                else:
                    print("robot doesnt mace moves")

                    ai_turn = True

            
            # TODO update current_positions according to the move
            # frame_time = time.time()
            # current_positions[next_move['label'], next_move['cell_index']] = frame_time

        if show_image:
            cv2.imshow('Real-Time Tic Tac Toe', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break  # Press 'q' to exit the loop

    video.release()
    cv2.destroyAllWindows()

X = "X"
O = "O"
EMPTY = None

def initial_state():
    """ Returns starting state of the board. """
    return [[EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY],
            [EMPTY, EMPTY, EMPTY]
            ]

def update_board(board, confirmed_moves):
    for move in confirmed_moves:
        row, col = move['cell_index'] // 3, move['cell_index'] % 3
        board[row][col] = X if move['label'] == "x" else O
    return board 

# Example usage


In [43]:
process_live_video(1, MODEL_PATH, show_image=False)


Could not find four corner points of the grid.
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}
[] {}


KeyboardInterrupt: 

In [44]:
import cv2

def display_live_camera():
    # Initialize the webcam. 0 is the default camera.
    cap = cv2.VideoCapture(1)

    if not cap.isOpened():
        print("Error: Could not open video capture.")
        return

    # Set the width and height if desired
    # cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    # cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

    print("Press 'q' to quit.")

    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()

        # if not ret:
        #     print("Failed to grab frame.")
        #     break

        # Optional: Flip the frame horizontally for a mirror effect
        # frame = cv2.flip(frame, 1)

        # Display the resulting frame
        cv2.imshow('Live Camera Feed', frame)

        # Wait for 1 ms and check if 'q' key is pressed to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Exiting...")
            break

    # When everything done, release the capture and close windows
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    display_live_camera()


Press 'q' to quit.
Exiting...
